In [1]:
# Master jupyter notebook for Case Study

In [2]:
# importing libraries
import pandas as pd
import sqlite3
from sklearn.preprocessing import LabelEncoder

In [3]:
# Extracting tables from database
def list_tables(database_file):
    connection = sqlite3.connect('data.db')
    cursor = connection.cursor()
    cursor.execute("""select name from sqlite_master where type = 'table';
                    """)
                    
    tables = cursor.fetchall()
    cursor.close()
    connection.close()
    table_names = [table[0] for table in tables]
    return table_names

# Saving tables as dataframes
connection = sqlite3.connect('data.db')
query = """select *
        from client as cl;
        """
client = pd.read_sql_query(query, connection)
connection.close

connection = sqlite3.connect('data.db')
query = """select *
        from client_products as cl;
        """
client_products = pd.read_sql_query(query, connection)
connection.close

connection = sqlite3.connect('data.db')
query = """select *
        from balances as cl;
        """
balances = pd.read_sql_query(query, connection)
connection.close

connection = sqlite3.connect('data.db')
query = """select *
        from inv_campaign_eval as cl;
        """
inv_campaign_eval = pd.read_sql_query(query, connection)
connection.close

# merging tables with client data (excluding balances and campaign eval table)
connection = sqlite3.connect('data.db')
query = """
        select cl.client_id, age, job, marital, education, gender, has_deposits, loan, has_insurance, has_mortgage
        from client as cl
        left join client_products as cp on cl.client_id = cp.client_id
        --left join balances as ba on cl.client_id = ba.client_id
        --left join inv_campaign_eval as camp on cl.client_id = camp.client_id;
        """
clients_merged = pd.read_sql_query(query, connection)
connection.close

# checking for datatypes
print(clients_merged.dtypes)

# checking for missing values
print(clients_merged.isnull().sum()) # missing values in Age and Job
print(balances.isna().sum()) # no missing values

client_id          int64
age              float64
job               object
marital           object
education         object
gender            object
has_deposits      object
loan              object
has_insurance     object
has_mortgage      object
dtype: object
client_id          0
age              232
job              438
marital            0
education          0
gender             0
has_deposits       0
loan               0
has_insurance      0
has_mortgage       0
dtype: int64
date         0
balance      0
client_id    0
currency     0
dtype: int64


In [4]:
# dealing with missing values
clients_merged['job'].fillna('unknown', inplace=True) # imputing "unknown" in job
clients_merged['age'].fillna(clients_merged['age'].median(), inplace=True) # imputing median in age

In [5]:
# label encoding categorical variables
le = LabelEncoder()
clients_merged['has_deposits_enc'] = le.fit_transform(clients_merged['has_deposits'])
clients_merged['loan_enc'] = le.fit_transform(clients_merged['loan'])
clients_merged['has_insurance_enc'] = le.fit_transform(clients_merged['has_insurance'])
clients_merged['has_mortgage_enc'] = le.fit_transform(clients_merged['has_mortgage'])
clients_merged['marital_enc'] = le.fit_transform(clients_merged['marital'])
clients_merged['education_enc'] = le.fit_transform(clients_merged['education'])
clients_merged['gender_enc'] = le.fit_transform(clients_merged['gender'])
clients_merged['job_enc'] = le.fit_transform(clients_merged['job'])

In [6]:
# extracting encoding labels
job_labels = clients_merged[['job', 'job_enc']].drop_duplicates().sort_values('job_enc')
gender_labels = clients_merged[['gender', 'gender_enc']].drop_duplicates().sort_values('gender_enc')
education_labels = clients_merged[['education', 'education_enc']].drop_duplicates().sort_values('education_enc')
marital_labels = clients_merged[['marital', 'marital_enc']].drop_duplicates().sort_values('marital_enc')
has_mortgage_labels = clients_merged[['has_mortgage', 'has_mortgage_enc']].drop_duplicates().sort_values('has_mortgage_enc')
has_insurance_labels = clients_merged[['has_insurance', 'has_insurance_enc']].drop_duplicates().sort_values('has_insurance_enc')
loan_labels = clients_merged[['loan', 'loan_enc']].drop_duplicates().sort_values('loan_enc')
has_deposits_labels = clients_merged[['has_deposits', 'has_deposits_enc']].drop_duplicates().sort_values('has_deposits_enc')

In [7]:
clients_merged

,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,has_deposits_enc,loan_enc,has_insurance_enc,has_mortgage_enc,marital_enc,education_enc,gender_enc,job_enc
0,249789938,38.0,services,married,secondary,M,yes,no,n,no,1,0,0,0,1,1,1,7
1,1222646323,46.0,services,divorced,unknown,M,no,no,n,no,0,0,0,0,0,3,1,7
2,451375919,33.0,admin.,single,secondary,F,no,no,n,yes,0,0,0,1,2,1,0,0
3,338972671,44.0,self-employed,married,secondary,F,no,no,y,yes,0,0,1,1,1,1,0,6
4,1472834688,36.0,blue-collar,married,primary,M,yes,no,n,yes,1,0,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,543198889,40.0,blue-collar,single,unknown,M,no,no,n,yes,0,0,0,1,2,3,1,1
11158,458558558,41.0,housemaid,divorced,primary,F,no,no,n,yes,0,0,0,1,0,0,0,3
11159,1426334329,59.0,technician,married,primary,F,no,no,y,yes,0,0,1,1,1,0,0,9
11160,831035343,42.0,management,married,tertiary,M,no,no,n,no,0,0,0,0,1,2,1,4


In [8]:
# feature engineering: balances
# last balance, mean balance, min balance, max balance
balances['date'] = pd.to_datetime(balances['date'])

# last balance
last_timestamp_index = balances.groupby('client_id')['date'].idxmax()
last_timestamp_df = balances.loc[last_timestamp_index]
last_timestamp_df.rename(columns={'balance': 'last_balance'}, inplace=True)
clients_merged = pd.merge(clients_merged, last_timestamp_df[['last_balance','client_id']], on = 'client_id')

# mean balance
mean_bal_by_client = balances.groupby('client_id')['balance'].mean().reset_index()
mean_bal_by_client.rename(columns={'balance': 'mean_balance'}, inplace=True)
clients_merged = pd.merge(clients_merged, mean_bal_by_client[['mean_balance','client_id']], on = 'client_id')

# min balance
min_bal_by_client = balances.groupby('client_id')['balance'].min().reset_index()
min_bal_by_client.rename(columns={'balance': 'min_balance'}, inplace=True)
clients_merged = pd.merge(clients_merged, min_bal_by_client[['min_balance','client_id']], on = 'client_id')

# max balance
max_bal_by_client = balances.groupby('client_id')['balance'].max().reset_index()
max_bal_by_client.rename(columns={'balance': 'max_balance'}, inplace=True)
clients_merged = pd.merge(clients_merged, max_bal_by_client[['max_balance','client_id']], on = 'client_id')


In [9]:
clients_merged

,client_id,age,job,marital,education,gender,has_deposits,loan,has_insurance,has_mortgage,...,has_insurance_enc,has_mortgage_enc,marital_enc,education_enc,gender_enc,job_enc,last_balance,mean_balance,min_balance,max_balance
0,249789938,38.0,services,married,secondary,M,yes,no,n,no,...,0,0,1,1,1,7,2680.467773,2669.532734,2515.500059,2775.615374
1,1222646323,46.0,services,divorced,unknown,M,no,no,n,no,...,0,0,0,3,1,7,1005.302876,960.562073,886.126906,1054.517907
2,451375919,33.0,admin.,single,secondary,F,no,no,n,yes,...,0,1,2,1,0,0,1100.917203,1221.016419,1100.917203,1303.494818
3,338972671,44.0,self-employed,married,secondary,F,no,no,y,yes,...,1,1,1,1,0,6,357.676547,297.993265,202.053088,446.676191
4,1472834688,36.0,blue-collar,married,primary,M,yes,no,n,yes,...,0,1,1,0,1,1,1906.382731,1919.318145,1853.387429,2011.939205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,543198889,40.0,blue-collar,single,unknown,M,no,no,n,yes,...,0,1,2,3,1,1,5.538113,18.441905,-98.265981,125.568092
11158,458558558,41.0,housemaid,divorced,primary,F,no,no,n,yes,...,0,1,0,0,0,3,70.463701,25.552707,-117.991255,116.816943
11159,1426334329,59.0,technician,married,primary,F,no,no,y,yes,...,1,1,1,0,0,9,3232.008038,3157.720202,3024.384824,3263.371390
11160,831035343,42.0,management,married,tertiary,M,no,no,n,no,...,0,0,1,2,1,4,24693.396851,24771.387533,24678.831382,24872.197971


In [11]:
# joining evaluation table with cleaned client table
first_round_clients = pd.merge(inv_campaign_eval, clients_merged, on = 'client_id')
first_round_clients

,client_id,poutcome,age,job,marital,education,gender,has_deposits,loan,has_insurance,...,has_insurance_enc,has_mortgage_enc,marital_enc,education_enc,gender_enc,job_enc,last_balance,mean_balance,min_balance,max_balance
0,249789938,success,38.0,services,married,secondary,M,yes,no,n,...,0,0,1,1,1,7,2680.467773,2669.532734,2515.500059,2775.615374
1,1504633819,failure,38.0,management,married,tertiary,M,no,yes,n,...,0,1,1,2,1,4,5476.241582,5440.862334,5344.385346,5529.866975
2,1952195738,success,34.0,services,married,secondary,M,yes,yes,n,...,0,0,1,1,1,7,2935.906506,2922.958852,2852.824056,3016.314818
3,989726384,success,62.0,retired,married,secondary,F,yes,no,n,...,0,0,1,1,0,5,3042.641066,3124.614843,3011.445910,3212.174671
4,1971878545,failure,36.0,management,single,tertiary,M,yes,no,n,...,0,0,2,2,1,4,526.157661,474.083033,362.809777,571.028715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2294,1270855655,failure,58.0,admin.,divorced,secondary,F,yes,no,y,...,1,1,0,1,0,0,3353.741381,3349.398115,3245.043023,3445.072323
2295,471724023,success,72.0,retired,married,primary,M,yes,no,n,...,0,0,1,0,1,5,4617.207488,4645.473515,4493.914363,4736.613098
2296,620088171,failure,34.0,admin.,married,tertiary,F,yes,no,y,...,1,1,1,2,0,0,939.603482,895.455390,771.707441,979.161966
2297,1518319265,success,39.0,management,married,unknown,F,yes,no,n,...,0,0,1,3,0,4,10.524798,77.494621,-36.431915,226.953215
